In [ ]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

In [ ]:
class Clustering:
    def __init__(self, space_size=400, r_sen=50, max_cluster_size=20, min_cluster_size=5):
        self.space_size = space_size
        self.r_sen = r_sen
        self.max_cluster_size = max_cluster_size
        self.min_cluster_size = min_cluster_size

    def estimate_optimal_k(self, nodes, base_station=(200,200,400)):
        """
        Ước tính số cụm tối ưu dựa trên công thức WSN
        K = sqrt(N*L / (pi*d_tobs))
        """
        N = len(nodes)
        base_pos = np.array(base_station)

        # Khoang cach trung binh toi base station
        distances = np.linalg.norm(nodes - base_pos, axis=1)
        d_tobs = np.mean(distances)

        space_size = self.space_size

        k_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
        k_optimal = max(2, int(np.round(k_optimal)))

        # Điều chỉnh dựa trên max_cluster_size
        k_min = int(np.ceil(N / self.max_cluster_size))
        k_optimal = max(k_optimal, k_min)
        
        return k_optimal
    
    def check_cluster_validity(self, cluster_nodes):
        """
        Kiem tra tinh hop le cua cum
        """
        size = len(cluster_nodes)

        # Kiểm tra kích thước
        if size < self.min_cluster_size or size > self.max_cluster_size:
            return False, 0, size
        
        # Kiểm tra khoảng cách
        if size > 1:
            distances = pdist(cluster_nodes)
            max_dist = np.max(distances)
            
            if max_dist > self.r_sen:
                return False, max_dist, size
            
            return True, max_dist, size
        
        return True, 0, size
    
    def split_invalid_cluster(self, cluster_nodes, cluster_ids):
        """
        Chia nhỏ cụm không hợp lệ thành các cụm con
        """
        # Nếu cụm chỉ có 1 node, không thể chia
        if len(cluster_nodes) < 2:
            return [(cluster_nodes, cluster_ids)]
        
        # Sử dụng K-Means để chia 2
        kmeans = KMeans(n_clusters=2, n_init=20, random_state=42)
        labels = kmeans.fit_predict(cluster_nodes)
        
        sub_clusters = []
        for i in range(2):
            sub_nodes = cluster_nodes[labels == i]
            sub_ids = [cluster_ids[j] for j in range(len(cluster_ids)) if labels[j] == i]
            
            if len(sub_nodes) > 0:
                sub_clusters.append((sub_nodes, sub_ids))
        
        return sub_clusters
    
    def merge_small_clusters(self, clusters_data):
        """
        Gộp các cụm nhỏ với cụm láng giềng gần nhất
        """
        if len(clusters_data) <= 1:
            return clusters_data
        
        merged = []
        to_merge = []
        
        # Tìm các cụm nhỏ
        for nodes, ids in clusters_data:
            if len(nodes) < self.min_cluster_size:
                to_merge.append((nodes, ids))
            else:
                merged.append((nodes, ids))
        
        # Gộp từng cụm nhỏ vào cụm gần nhất
        for small_nodes, small_ids in to_merge:
            if len(merged) == 0:
                merged.append((small_nodes, small_ids))
                continue
            
            # Tìm cụm gần nhất
            small_center = np.mean(small_nodes, axis=0)
            min_dist = float('inf')
            best_idx = 0
            
            for i, (nodes, ids) in enumerate(merged):
                center = np.mean(nodes, axis=0)
                dist = np.linalg.norm(small_center - center)
                
                # Kiểm tra xem gộp có vượt quá max_size không
                if dist < min_dist and len(nodes) + len(small_nodes) <= self.max_cluster_size:
                    min_dist = dist
                    best_idx = i
            
            # Gộp
            merged[best_idx] = (
                np.vstack([merged[best_idx][0], small_nodes]),
                merged[best_idx][1] + small_ids
            )
        
        return merged
    
    def cluster_with_constraints(self, nodes, node_ids, k=None, max_iterations=10):
        """
        Phân cụm với ràng buộc - Thuật toán chính
        
        Args:
            nodes: Tọa độ 3D của nodes
            node_ids: ID của nodes
            k: Số cụm (nếu None sẽ tự động ước tính)
            max_iterations: Số lần lặp tối đa để điều chỉnh
            
        Returns:
            List of (cluster_nodes, cluster_ids)
        """
        if k is None:
            k = self.estimate_optimal_k(nodes)
        
        print(f"Bắt đầu phân cụm với k={k}")
        
        # Bước 1: K-Means ban đầu
        kmeans = KMeans(n_clusters=k, n_init=30, random_state=42)
        labels = kmeans.fit_predict(nodes)
        
        # Bước 2: Tạo các cụm và kiểm tra
        iteration = 0
        while iteration < max_iterations:
            print(f"  Vòng lặp {iteration + 1}/{max_iterations}")
            
            valid_clusters = []
            invalid_clusters = []
            
            # Phân loại cụm hợp lệ và không hợp lệ
            for i in range(k):
                cluster_nodes = nodes[labels == i]
                cluster_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
                
                if len(cluster_nodes) == 0:
                    continue
                
                is_valid, max_dist, size = self.check_cluster_validity(cluster_nodes)
                
                if is_valid:
                    valid_clusters.append((cluster_nodes, cluster_ids))
                    print(f"    Cụm {i}: ✓ hợp lệ (size={size}, max_dist={max_dist:.1f}m)")
                else:
                    invalid_clusters.append((cluster_nodes, cluster_ids))
                    print(f"    Cụm {i}: ✗ không hợp lệ (size={size}, max_dist={max_dist:.1f}m)")
            
            # Nếu tất cả hợp lệ, kết thúc
            if len(invalid_clusters) == 0:
                print(f"  → Tất cả cụm hợp lệ!")
                break
            
            # Bước 3: Xử lý các cụm không hợp lệ
            for cluster_nodes, cluster_ids in invalid_clusters:
                size = len(cluster_nodes)
                
                if size > self.max_cluster_size:
                    # Cụm quá lớn → Chia nhỏ
                    print(f"    → Chia cụm (size={size})")
                    sub_clusters = self.split_invalid_cluster(cluster_nodes, cluster_ids)
                    valid_clusters.extend(sub_clusters)
                else:
                    # Cụm có khoảng cách quá lớn → Chia nhỏ
                    print(f"    → Chia cụm (khoảng cách lớn)")
                    sub_clusters = self.split_invalid_cluster(cluster_nodes, cluster_ids)
                    valid_clusters.extend(sub_clusters)
            
            # Cập nhật labels và k cho vòng lặp tiếp theo
            k = len(valid_clusters)
            
            # Tạo lại labels từ valid_clusters
            labels = np.zeros(len(nodes), dtype=int)
            for cluster_idx, (_, cluster_ids) in enumerate(valid_clusters):
                for node_id in cluster_ids:
                    node_idx = node_ids.index(node_id)
                    labels[node_idx] = cluster_idx
            
            iteration += 1
        
        # Bước 4: Gộp các cụm quá nhỏ
        valid_clusters = self.merge_small_clusters(valid_clusters)
        
        print(f"Hoàn thành: {len(valid_clusters)} cụm")
        return valid_clusters
    
    def choose_cluster_head(self, cluster_nodes, cluster_ids, node_data=None):
        """
        Chọn cluster head
        - Ưu tiên: Node có năng lượng cao nhất
        - Dự phòng: Node gần tâm cụm nhất
        """
        if node_data:
            # Chọn theo năng lượng
            max_energy = -1
            ch_id = cluster_ids[0]
            
            for nid in cluster_ids:
                if nid in node_data and 'residual_energy' in node_data[nid]:
                    energy = node_data[nid]['residual_energy']
                    if energy > max_energy:
                        max_energy = energy
                        ch_id = nid
            
            return ch_id
        else:
            # Chọn theo khoảng cách đến tâm
            center = np.mean(cluster_nodes, axis=0)
            distances = np.linalg.norm(cluster_nodes - center, axis=1)
            min_idx = np.argmin(distances)
            return cluster_ids[min_idx]
    
    def calculate_metrics(self, clusters_data):
        """
        Tính các metric đánh giá chất lượng phân cụm
        """
        metrics = {
            'num_clusters': len(clusters_data),
            'avg_cluster_size': 0,
            'min_cluster_size': float('inf'),
            'max_cluster_size': 0,
            'avg_intra_distance': 0,
            'max_intra_distance': 0,
            'balance_score': 0  # Độ cân bằng kích thước cụm (0-1, càng cao càng tốt)
        }
        
        sizes = []
        intra_dists = []
        
        for nodes, ids in clusters_data:
            size = len(nodes)
            sizes.append(size)
            
            metrics['min_cluster_size'] = min(metrics['min_cluster_size'], size)
            metrics['max_cluster_size'] = max(metrics['max_cluster_size'], size)
            
            if size > 1:
                distances = pdist(nodes)
                intra_dists.append(np.mean(distances))
                metrics['max_intra_distance'] = max(metrics['max_intra_distance'], np.max(distances))
        
        metrics['avg_cluster_size'] = np.mean(sizes)
        metrics['avg_intra_distance'] = np.mean(intra_dists) if intra_dists else 0
        
        # Tính balance score (dựa trên coefficient of variation)
        cv = np.std(sizes) / np.mean(sizes) if np.mean(sizes) > 0 else 0
        metrics['balance_score'] = 1 / (1 + cv)  # 1 = hoàn toàn cân bằng
        
        return metrics

def process_data(input_file, output_folder, draw_folder, 
                    r_sen=50, max_size=20, min_size=5):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    nodes = np.array([[d['x'], d['y'], d['z']] for d in data])
    node_ids = [d['id'] for d in data]
    
    # Tạo node_data với thông tin năng lượng
    node_data = {}
    for d in data:
        if 'residual_energy' in d:
            node_data[d['id']] = {
                'residual_energy': d['residual_energy'],
                'initial_energy': d.get('initial_energy', 100.0)
            }
    
    # Khởi tạo clustering
    clustering = Clustering(
        r_sen=r_sen,
        max_cluster_size=max_size,
        min_cluster_size=min_size
    )
    
    # Phân cụm
    print(f"\n{'='*60}")
    print(f"Xử lý: {os.path.basename(input_file)}")
    print(f"Số nodes: {len(nodes)}")
    print(f"Tham số: r_sen={r_sen}m, max_size={max_size}, min_size={min_size}")
    print(f"{'='*60}")
    
    clusters_data = clustering.cluster_with_constraints(nodes, node_ids)
    
    # Tính metrics
    metrics = clustering.calculate_metrics(clusters_data)
    
    print(f"\n{'='*60}")
    print("KẾT QUẢ:")
    print(f"  Số cụm: {metrics['num_clusters']}")
    print(f"  Kích thước trung bình: {metrics['avg_cluster_size']:.1f}")
    print(f"  Kích thước: [{metrics['min_cluster_size']} - {metrics['max_cluster_size']}]")
    print(f"  Khoảng cách trung bình trong cụm: {metrics['avg_intra_distance']:.1f}m")
    print(f"  Khoảng cách max trong cụm: {metrics['max_intra_distance']:.1f}m")
    print(f"  Độ cân bằng: {metrics['balance_score']:.2%}")
    print(f"{'='*60}\n")
    
    # Tạo output
    output_data = {}
    for i, (cluster_nodes, cluster_ids) in enumerate(clusters_data):
        ch = clustering.choose_cluster_head(cluster_nodes, cluster_ids, node_data)
        center = np.mean(cluster_nodes, axis=0)
        
        output_data[i] = {
            'nodes': cluster_ids,
            'center': [float(x) for x in np.round(center, 2)],
            'cluster_head': int(ch),
            'size': len(cluster_ids)
        }
    
    # Lưu output
    output_file = os.path.join(output_folder, os.path.basename(input_file))
    with open(output_file, 'w') as f:
        json.dump(output_data, f, indent=4)
    print(f"✓ Đã lưu: {output_file}")
    
    # Vẽ biểu đồ
    draw_file = os.path.join(draw_folder, 
                            os.path.basename(input_file).replace('.json', '.png'))
    visualize_clusters(nodes, clusters_data, output_data, draw_file)
    print(f"✓ Đã vẽ: {draw_file}")
    
    return output_data, metrics

def visualize_clusters(nodes, clusters_data, output_data, save_path):
    """
    Vẽ biểu đồ 3D các cụm
    """
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    num_clusters = len(clusters_data)
    cmap = plt.colormaps.get_cmap('tab20' if num_clusters > 10 else 'tab10')
    
    for i, (cluster_nodes, cluster_ids) in enumerate(clusters_data):
        color = cmap(i % 20)
        
        # Vẽ nodes
        ax.scatter(cluster_nodes[:, 0], cluster_nodes[:, 1], cluster_nodes[:, 2],
                  label=f'Cụm {i} ({len(cluster_ids)})',
                  color=color, alpha=0.6, s=50)
        
        # Vẽ cluster head
        ch_id = output_data[i]['cluster_head']
        ch_idx = cluster_ids.index(ch_id)
        ch_pos = cluster_nodes[ch_idx]
        ax.scatter(ch_pos[0], ch_pos[1], ch_pos[2],
                  color=color, marker='*', s=400, 
                  edgecolor='black', linewidth=1.5, zorder=100)
        
        # Vẽ tâm cụm
        center = output_data[i]['center']
        ax.scatter(center[0], center[1], center[2],
                  color=color, marker='x', s=100, linewidth=2, zorder=90)
    
    # Vẽ base station
    ax.scatter(0, 0, 0, color='red', marker='^', s=500,
              label='Base Station', edgecolor='black', linewidth=2, zorder=110)
    
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)
    ax.set_zlabel('Z (m)', fontsize=11)
    ax.set_title(f'WSN Clustering - {len(nodes)} nodes, {num_clusters} cụm',
                fontsize=13, fontweight='bold')
    
    if num_clusters <= 15:
        ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=9)
    
    ax.view_init(elev=25, azim=45)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=100, bbox_inches='tight')
    plt.close(fig)


# ===== MAIN EXECUTION =====
if __name__ == "__main__":
    # Tạo thư mục output
    input_folder = "IT4906/input_data"
    output_folder = "output_data_optimized"
    draw_folder = "draw_output_optimized"
    
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(draw_folder, exist_ok=True)
    
    # Tham số
    R_SEN = 60 # Bán kính truyền tải (m)
    MAX_SIZE = 20  # Kích thước cụm tối đa
    MIN_SIZE = 5  # Kích thước cụm tối thiểu
    
    # Xử lý từng file
    all_metrics = {}
    
    for filename in sorted(os.listdir(input_folder)):
        if filename.startswith("nodes_") and filename.endswith(".json"):
            input_path = os.path.join(input_folder, filename)
            
            try:
                output_data, metrics = process_data(
                    input_path, output_folder, draw_folder,
                    r_sen=R_SEN, max_size=MAX_SIZE, min_size=MIN_SIZE
                )
                all_metrics[filename] = metrics
            except Exception as e:
                print(f"✗ Lỗi xử lý {filename}: {e}")
                continue
    
    # Tổng kết
    print("\n" + "="*60)
    print("TỔNG KẾT TOÀN BỘ")
    print("="*60)
    for filename, metrics in all_metrics.items():
        print(f"\n{filename}:")
        print(f"  Cụm: {metrics['num_clusters']}, "
              f"Size: {metrics['avg_cluster_size']:.1f} ± "
              f"{metrics['max_cluster_size']-metrics['min_cluster_size']}, "
              f"Balance: {metrics['balance_score']:.2%}")


In [ ]:
def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    if abs(cos_beta) < 1e-9:
        return v_AUV
    return abs(math.cos(angle) * v_AUV / cos_beta)

def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    if len(path) <= 1:
        return 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # return to start
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time

In [ ]:
class ClusterTSP_GA:
    """GA tối ưu hóa tour qua các cluster centers, sử dụng travel_time/compute_vs để tính mục tiêu (thời gian)."""
    def __init__(self, clusters, ga_params=None):
        # clusters: dict mapping cluster_id -> {"center": (x,y,z), ...}
        self.clusters = clusters
        # tạo list centers với điểm bắt đầu ở index 0
        self.cluster_centers = [(0.0, 0.0, 0.0)] + [clusters[k]["center"] for k in sorted(clusters.keys())]
        self.n = len(self.cluster_centers)

        # default params
        defaults = {
            'pop_size': 50,
            'generations': 200,
            'crossover_rate': 0.8,
            'mutation_rate': 0.2,
            'elitism_k': 3,
            'tournament_size': 3,
            'crossover_type': 'OX',
            'mutation_type': 'inversion',
            'local_search': True,
            'v_f': 0.3,
            'v_AUV': 1.0,
            'verbose': False
        }
        if ga_params:
            defaults.update(ga_params)
        self.params = defaults

        self.best_fitness_history = []
        self.avg_fitness_history = []

    def create_individual(self):
        seq = list(range(1, self.n))
        random.shuffle(seq)
        return [0] + seq

    def create_population(self):
        pop = []
        # add a nearest neighbor as seed
        pop.append(self.nearest_neighbor(0))
        while len(pop) < self.params['pop_size']:
            pop.append(self.create_individual())
        return pop

    def nearest_neighbor(self, start=0):
        unvisited = set(range(1, self.n))
        tour = [start]
        cur = start
        while unvisited:
            nxt = min(unvisited, key=lambda x: np.linalg.norm(np.array(self.cluster_centers[cur]) - np.array(self.cluster_centers[x])))
            tour.append(nxt)
            unvisited.remove(nxt)
            cur = nxt
        return tour

    def fitness(self, individual):
        # fitness = 1 / total_time
        total_time = travel_time(individual, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        return 1.0 / (total_time + 1e-9)

    def tournament_selection(self, population):
        candidates = random.sample(population, self.params['tournament_size'])
        return max(candidates, key=self.fitness)

    def order_crossover(self, p1, p2):
        # OX on subarray excluding index 0
        sub1 = p1[1:]
        sub2 = p2[1:]
        m = len(sub1)
        if m < 2:
            return p1.copy(), p2.copy()
        a, b = sorted(random.sample(range(m), 2))
        c1 = [-1]*m
        c2 = [-1]*m
        c1[a:b] = sub1[a:b]
        ptr = b
        for x in sub2[b:]+sub2[:b]:
            if x not in c1:
                c1[ptr % m] = x
                ptr += 1
        c2[a:b] = sub2[a:b]
        ptr = b
        for x in sub1[b:]+sub1[:b]:
            if x not in c2:
                c2[ptr % m] = x
                ptr += 1
        return [0]+c1, [0]+c2

    def pmx_crossover(self, p1, p2):
        sub1 = p1[1:]
        sub2 = p2[1:]
        m = len(sub1)
        if m < 2:
            return p1.copy(), p2.copy()
        a, b = sorted(random.sample(range(m), 2))
        c1 = [-1]*m
        c2 = [-1]*m
        c1[a:b] = sub1[a:b]
        c2[a:b] = sub2[a:b]
        mapping1 = {sub1[i]: sub2[i] for i in range(a,b)}
        mapping2 = {sub2[i]: sub1[i] for i in range(a,b)}
        for i in list(range(0,a)) + list(range(b,m)):
            val = sub2[i]
            while val in mapping1:
                val = mapping1[val]
            c1[i] = val
            val = sub1[i]
            while val in mapping2:
                val = mapping2[val]
            c2[i] = val
        return [0]+c1, [0]+c2

    def swap_mutation(self, ind):
        ind = ind.copy()
        if len(ind) > 2:
            i, j = random.sample(range(1, len(ind)), 2)
            ind[i], ind[j] = ind[j], ind[i]
        return ind

    def inversion_mutation(self, ind):
        ind = ind.copy()
        if len(ind) > 2:
            i, j = sorted(random.sample(range(1, len(ind)), 2))
            ind[i:j+1] = list(reversed(ind[i:j+1]))
        return ind

    def two_opt(self, tour):
        improved = True
        best = tour.copy()
        best_time = travel_time(best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        while improved:
            improved = False
            for i in range(1, len(best)-2):
                for j in range(i+1, len(best)-1):
                    cand = best.copy()
                    cand[i:j+1] = list(reversed(cand[i:j+1]))
                    t = travel_time(cand, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
                    if t < best_time:
                        best = cand
                        best_time = t
                        improved = True
                        break
                if improved:
                    break
        return best

    def evolve(self):
        pop = self.create_population()
        best = max(pop, key=self.fitness)
        best_time = travel_time(best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
        for gen in range(self.params['generations']):
            fitnesses = [self.fitness(ind) for ind in pop]
            self.best_fitness_history.append(max(fitnesses))
            self.avg_fitness_history.append(float(np.mean(fitnesses)))
            gen_best = pop[np.argmax(fitnesses)]
            gen_best_time = travel_time(gen_best, self.cluster_centers, self.params['v_f'], self.params['v_AUV'])
            if gen_best_time < best_time:
                best = gen_best.copy()
                best_time = gen_best_time
                if self.params['verbose'] and gen % 50 == 0:
                    print(f"   Gen {gen}: new best time = {best_time:.4f} s")

            # elite
            elite_idx = np.argsort(fitnesses)[-self.params['elitism_k']:]
            new_pop = [pop[i].copy() for i in elite_idx]
            while len(new_pop) < self.params['pop_size']:
                p1 = self.tournament_selection(pop)
                p2 = self.tournament_selection(pop)
                if random.random() < self.params['crossover_rate']:
                    if self.params['crossover_type'] == 'OX':
                        c1, c2 = self.order_crossover(p1, p2)
                    else:
                        c1, c2 = self.pmx_crossover(p1, p2)
                else:
                    c1, c2 = p1.copy(), p2.copy()

                if random.random() < self.params['mutation_rate']:
                    if self.params['mutation_type'] == 'swap':
                        c1 = self.swap_mutation(c1)
                    else:
                        c1 = self.inversion_mutation(c1)
                if random.random() < self.params['mutation_rate']:
                    if self.params['mutation_type'] == 'swap':
                        c2 = self.swap_mutation(c2)
                    else:
                        c2 = self.inversion_mutation(c2)

                if self.params['local_search'] and random.random() < 0.1:
                    c1 = self.two_opt(c1)
                    c2 = self.two_opt(c2)

                new_pop.extend([c1, c2])

            pop = new_pop[:self.params['pop_size']]

        return best, best_time

In [ ]:
def compute_energy(best_time, n_members):
    """
    Tính năng lượng tiêu thụ cho Member Node và Cluster Head.
    
    Parameters:
    - best_time: Thời gian hoàn thành chu kỳ AUV
    - n_members: Số lượng node thành viên thực tế trong cluster (không tính cluster head)
    """
    G, L = 100, 1024
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6

    # Năng lượng cho Member Node
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN

    # Năng lượng cho Cluster Head (nhận từ n_members node, truyền cho AUV)
    E_rx_TN = G * P_r * L * n_members / DR
    E_tx_TN = G * P_t * L * n_members / DR_i
    E_idle_TN = (best_time - (G*L*n_members/DR) - (G*L*n_members/DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN

    return {
        "Member": {"E_total": E_total_MN},
        "Target": {"E_total": E_total_TN}
    }

def update_energy(all_nodes, clusters, best_time):
    """
    Cập nhật năng lượng cho tất cả các node dựa trên số member thực tế của từng cluster.
    
    Parameters:
    - all_nodes: Dictionary chứa thông tin tất cả các node
    - clusters: Dictionary chứa thông tin các cluster
    - best_time: Thời gian hoàn thành chu kỳ AUV
    """
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        
        # Tính số member nodes (không tính cluster head)
        n_members = len([n for n in nodes if n != ch])
        
        # Tính năng lượng cho cluster này với số member thực tế
        energy_report = compute_energy(best_time, n_members)
        
        for nid in nodes:
            if nid not in all_nodes: continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            all_nodes[nid]['residual_energy'] = max(all_nodes[nid]['residual_energy'], 0.0)

def remove_dead_nodes(all_nodes, clusters):
    """
    Loại bỏ các node đã hết năng lượng và cập nhật lại clusters.
    
    Returns:
    - new_clusters: Dictionary các cluster còn node sống
    - dead: List các node_id đã chết
    """
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]

    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            new_clusters[cid] = new_c

    return new_clusters, dead

In [ ]:
def ga_path_optimization(clusters, v_f, v_AUV, verbose=False):
    """
    Sử dụng Genetic Algorithm để tìm đường đi tối ưu qua các cluster.
    
    Parameters:
    - clusters: Dictionary các cluster với center
    - v_f: Vận tốc dòng chảy
    - v_AUV: Vận tốc AUV
    - verbose: In thông tin tiến trình GA
    
    Returns:
    - path_indices: List các index theo thứ tự thăm
    - best_time: Thời gian tối ưu
    """
    if len(clusters) == 0:
        return [0], 0.0
    
    ga_params = {
        'pop_size': 40,
        'generations': 150,
        'crossover_rate': 0.8,
        'mutation_rate': 0.2,
        'elitism_k': 3,
        'local_search': True,
        'v_f': v_f,
        'v_AUV': v_AUV,
        'verbose': verbose
    }
    
    ga = ClusterTSP_GA(clusters, ga_params)
    best_path, best_time = ga.evolve()
    
    return best_path, best_time

def recluster(all_nodes, node_positions, clustering_instance, r_sen=60, max_size=20, min_size=5):
    """
    Phân cụm lại toàn bộ các node còn sống sử dụng thuật toán từ cluster.py.
    
    Parameters:
    - all_nodes: Dictionary các node còn sống
    - node_positions: Dictionary vị trí của các node
    - clustering_instance: Instance của class Clustering
    - r_sen: Ngưỡng khoảng cách tối đa trong cụm
    - max_size: Số lượng node tối đa trong 1 cụm
    - min_size: Số lượng node tối thiểu trong 1 cụm
    
    Returns:
    - clusters: Dictionary các cụm mới
    """
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}

    # Tạo mảng tọa độ nodes
    coords = np.array([node_positions[nid] for nid in ids])
    
    # Cập nhật tham số cho clustering instance
    clustering_instance.r_sen = r_sen
    clustering_instance.max_cluster_size = max_size
    clustering_instance.min_cluster_size = min_size
    
    # Phân cụm với ràng buộc
    clusters_data = clustering_instance.cluster_with_constraints(coords, ids)
    
    # Chuyển đổi sang format cần thiết
    clusters = {}
    for i, (cluster_nodes, cluster_ids) in enumerate(clusters_data):
        center = np.mean(cluster_nodes, axis=0).tolist()
        
        # Chọn cluster head
        ch = clustering_instance.choose_cluster_head(cluster_nodes, cluster_ids, all_nodes)
        
        clusters[i] = {
            'nodes': cluster_ids,
            'center': center,
            'cluster_head': ch
        }
    
    return clusters

In [ ]:
def main():
    """
    Hàm chính mô phỏng mạng cảm biến dưới nước với AUV thu thập dữ liệu.
    Sử dụng Genetic Algorithm để tối ưu đường đi.
    """
    input_dir = "input_data"
    output_dir = "result_ga_ch_most_energy"
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_dir):
        print(f"❌ Lỗi: Thư mục {input_dir} không tồn tại!")
        return

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
    
    if len(files) == 0:
        print(f"❌ Không tìm thấy file dữ liệu nào trong {input_dir}")
        return

    # Tham số
    INITIAL_ENERGY = 100.0
    v_f = 1.2
    v_AUV = 3.0
    R_SEN = 60      # Bán kính truyền tải (m)
    MAX_SIZE = 20   # Kích thước cụm tối đa
    MIN_SIZE = 5    # Kích thước cụm tối thiểu
    
    results_summary = []
    
    # Khởi tạo Clustering instance
    clustering = Clustering(
        space_size=400,
        r_sen=R_SEN,
        max_cluster_size=MAX_SIZE,
        min_cluster_size=MIN_SIZE
    )

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n{'='*60}")
        print(f"=== Đang xử lý file: {filename} ===")
        print(f"{'='*60}")
        
        try:
            with open(input_path, 'r') as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Lỗi đọc file {filename}: {e}")
            continue

        node_positions = {}
        all_nodes = {}
        
        # Xử lý file JSON - danh sách nodes: [{"id": 0, "x": ..., "y": ..., "z": ...}, ...]
        if isinstance(data, list):
            print("Định dạng: Danh sách nodes")
            for node in data:
                nid = node['id']
                all_nodes[nid] = {
                    'initial_energy': node.get('initial_energy', INITIAL_ENERGY),
                    'residual_energy': node.get('residual_energy', INITIAL_ENERGY)
                }
                node_positions[nid] = (node['x'], node['y'], node['z'])
        else:
            print(f"❌ Cấu trúc file {filename} không được hỗ trợ (cần list of nodes)")
            continue

        total_nodes = len(all_nodes)
        print(f"Tổng số node ban đầu: {total_nodes}")
        print(f"Tham số: r_sen={R_SEN}m, max_size={MAX_SIZE}, min_size={MIN_SIZE}")
        print(f"Phương pháp: Genetic Algorithm (GA)")

        # Phân cụm lần đầu tiên
        print(f"\n{'='*60}")
        print("PHÂN CỤM LẦN ĐẦU TIÊN")
        print(f"{'='*60}")
        initial_clusters = recluster(all_nodes, node_positions, clustering, R_SEN, MAX_SIZE, MIN_SIZE)
        
        # Tính metrics cho phân cụm ban đầu
        clusters_data_for_metrics = []
        for cid, cinfo in initial_clusters.items():
            cluster_nodes = np.array([node_positions[nid] for nid in cinfo['nodes']])
            clusters_data_for_metrics.append((cluster_nodes, cinfo['nodes']))
        
        metrics = clustering.calculate_metrics(clusters_data_for_metrics)
        
        print(f"\n METRICS PHÂN CỤM:")
        print(f"  - Số cụm: {metrics['num_clusters']}")
        print(f"  - Kích thước TB: {metrics['avg_cluster_size']:.1f}")
        print(f"  - Kích thước: [{metrics['min_cluster_size']} - {metrics['max_cluster_size']}]")
        print(f"  - Khoảng cách TB trong cụm: {metrics['avg_intra_distance']:.1f}m")
        print(f"  - Khoảng cách max trong cụm: {metrics['max_intra_distance']:.1f}m")
        print(f"  - Độ cân bằng: {metrics['balance_score']:.2%}")
        
        # Lưu kết quả phân cụm lần đầu
        clusters_output = {}
        for cid, cinfo in initial_clusters.items():
            clusters_output[cid] = {
                'cluster_id': cid,
                'nodes': cinfo['nodes'],
                'center': cinfo['center'],
                'cluster_head': cinfo['cluster_head'],
                'num_nodes': len(cinfo['nodes'])
            }
        
        initial_cluster_file = os.path.join(output_dir, f"initial_clusters_{filename}")
        with open(initial_cluster_file, "w", encoding='utf-8') as f:
            json.dump(clusters_output, f, indent=4, ensure_ascii=False)
        
        print(f"\n Đã lưu phân cụm ban đầu: {initial_cluster_file}")

        cycle = 0
        alive_log = []
        energy_log = []
        cluster_count_log = []

        # Vòng lặp mô phỏng
        print(f"\n{'='*60}")
        print(" BẮT ĐẦU MÔ PHỎNG (sử dụng GA)")
        print(f"{'='*60}")
        
        while True:
            cycle += 1
            alive_log.append(len(all_nodes))
            total_energy = sum(all_nodes[n]['residual_energy'] for n in all_nodes)
            energy_log.append(total_energy)

            alive_ratio = len(all_nodes)/total_nodes if total_nodes > 0 else 0
            
            if alive_ratio < 0.9:
                print(f"\n🛑 Dừng mô phỏng ở cycle {cycle}: < 90% node còn sống ({alive_ratio*100:.2f}%)")
                break

            # Phân cụm lại
            clusters = recluster(all_nodes, node_positions, clustering, R_SEN, MAX_SIZE, MIN_SIZE)
            cluster_count_log.append(len(clusters))
            
            if len(clusters) == 0:
                print(f"\n Dừng mô phỏng ở cycle {cycle}: Không còn node")
                break

            print(f"\n--- Cycle {cycle} --- | Alive: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes}) | Energy: {total_energy:.2f}J | Clusters: {len(clusters)}")

            # Tạo đường đi cho AUV bằng GA (thay vì Greedy)
            print(f"   🧬 Chạy GA để tối ưu đường đi...")
            path_indices, best_time = ga_path_optimization(clusters, v_f, v_AUV, verbose=False)
            print(f"   ✅ GA hoàn thành: Best time = {best_time:.2f}s")
            
            # Cập nhật năng lượng
            update_energy(all_nodes, clusters, best_time)
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            
            if dead_nodes:
                print(f"   ⚡ {len(dead_nodes)} node(s) đã hết năng lượng")

        # Lưu kết quả JSON
        meta = {
            'input_file': filename,
            'initial_total_nodes': total_nodes,
            'cycles_completed': cycle - 1,
            'final_alive_nodes': len(all_nodes),
            'final_alive_ratio': len(all_nodes)/total_nodes if total_nodes > 0 else 0,
            'method': 'Genetic Algorithm (GA)',
            'parameters': {
                'r_sen': R_SEN,
                'max_cluster_size': MAX_SIZE,
                'min_cluster_size': MIN_SIZE,
                'v_flow': v_f,
                'v_AUV': v_AUV,
                'ga_pop_size': 40,
                'ga_generations': 150
            },
            'initial_clustering_metrics': {
                'num_clusters': metrics['num_clusters'],
                'avg_cluster_size': float(metrics['avg_cluster_size']),
                'balance_score': float(metrics['balance_score'])
            }
        }
        
        output_json = os.path.join(output_dir, f"result_{filename}")
        with open(output_json, "w") as f:
            json.dump(meta, f, indent=4)

        results_summary.append((filename, cycle - 1))
        print(f"\n File {filename}: {cycle - 1} cycles hoàn thành")

        # Plot 1: Alive nodes per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(alive_log)), alive_log, marker='o', linewidth=2, color='steelblue')
        plt.title(f"Số node sống theo chu kỳ - {filename} (GA)", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Nodes alive", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.axhline(y=total_nodes*0.9, color='red', linestyle='--', linewidth=2, label='Ngưỡng 90%')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"alive_nodes_{filename}.png"), dpi=150)
        plt.close()

        # Plot 2: Total energy per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(energy_log)), energy_log, marker='s', linewidth=2, color='orange')
        plt.title(f"Năng lượng toàn mạng theo chu kỳ - {filename} (GA)", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Total energy (J)", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"energy_{filename}.png"), dpi=150)
        plt.close()
        
        # Plot 3: Number of clusters per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(cluster_count_log)+1), cluster_count_log, marker='^', linewidth=2, color='green')
        plt.title(f"Số cụm theo chu kỳ - {filename} (GA)", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Số cụm", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"clusters_{filename}.png"), dpi=150)
        plt.close()

    # Summary plot
    if results_summary:
        labels = [x[0] for x in results_summary]
        values = [x[1] for x in results_summary]
        plt.figure(figsize=(10, 6))
        plt.plot(labels, values, marker='o', linewidth=2, markersize=8)
        plt.title("AUV cycles completed per dataset (GA)", fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel("Cycles", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "summary_cycles.png"), dpi=150)
        plt.close()
        
        print(f"\n{'='*60}")
        print(f"✅ Hoàn thành! Kết quả đã lưu tại: {output_dir}")
        print(f"{'='*60}")
    else:
        print("\n  Không có kết quả nào được tạo ra")
